In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

gpg: keybox '/tmp/tmp08imy6pd/pubring.gpg' created
gpg: /tmp/tmp08imy6pd/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
folder='./drive/ML/Cifar-10/cifar-10-batches-py'

In [0]:
import pandas as pd
import numpy as np
import os
import pickle


In [0]:
def unpickle(file):
  
  '''Load byte data from file'''
  with open(file, 'rb') as f:
    data = pickle.load(f)
    return data


def load_cifar10_data(data_dir):
  '''Return train_data, train_labels, test_data, test_labels
   The shape of data is 32 x 32 x3'''
    
  train_data = None
  train_labels = []

  for i in range(1, 6):
    data_dic = unpickle(data_dir + "/data_batch_{}".format(i))
    if i == 1:
      train_data = data_dic['data']
    else:
      train_data = np.vstack((train_data, data_dic['data']))
    train_labels += data_dic['labels']
  test_data_dic = unpickle(data_dir + "/test_batch")
  test_data = test_data_dic['data']
  test_labels = test_data_dic['labels']

  train_data = train_data.reshape((len(train_data), 3, 32, 32))
  train_data = np.rollaxis(train_data, 1, 4)
  train_labels = np.array(train_labels)

  test_data = test_data.reshape((len(test_data), 3, 32, 32))
  test_data = np.rollaxis(test_data, 1, 4)
  test_labels = np.array(test_labels)

  return train_data, train_labels, test_data, test_labels

In [0]:
train_data, train_labels, test_data, test_labels = load_cifar10_data(folder)


In [7]:
print(train_data.shape)
print(train_labels.shape)

print(test_data.shape)
print(test_labels.shape)


(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [8]:
import keras

Using TensorFlow backend.


In [0]:
y_train=np.asarray(train_labels).reshape(-1,1)
y_train=keras.utils.to_categorical(y_train,num_classes=10)

y_test=np.asarray(test_labels).reshape(-1,1)
y_test=keras.utils.to_categorical(y_test,num_classes=10)



In [0]:
x_train=train_data.astype('float')
x_test=test_data.astype('float')
x_train/=255.0
x_test/=255.0

In [0]:
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,Input,AveragePooling2D,BatchNormalization,Activation,GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy,categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator

In [12]:
datagen=ImageDataGenerator(featurewise_std_normalization=True,horizontal_flip=True)
datagen.fit(x_train)
train_gen=datagen.flow(x_train,y_train,batch_size=128,shuffle=True)
test_gen=datagen.flow(x_test,y_test,batch_size=128,shuffle=True)

/usr/local/lib/python2.7/dist-packages/keras/preprocessing/image.py:657: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
def residual_layer(y,channels,_strides=(1,1),project_shortcut=True):
    shortcut=y
    y=Conv2D(channels,kernel_size=(3,3),strides=_strides,activation='relu',padding='same')(y)
    y=BatchNormalization()(y)
    y=Activation('relu')(y)
    
    y=Conv2D(channels,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(y)
    y=BatchNormalization()(y)
    y=Activation('relu')(y)
    
    if(_strides!=(1,1)):
        shortcut=Conv2D(channels,kernel_size=(1,1),strides=_strides,activation='relu',padding='same')(shortcut)
        shortcut=BatchNormalization()(shortcut)
        
    y=keras.layers.add([shortcut,y])
    y=Activation('relu')(y)
    return y
    
    
    
    
    

In [0]:
def make_model(classes=10):
    inputs=Input(shape=(32,32,3))
    l=Conv2D(16,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(inputs)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    
    l1=l
    
    for i in range(18):
      l1=residual_layer(l1,16)
      
    l2=residual_layer(l1,32,(2,2))
    for i in range(17):
      l2=residual_layer(l2,32)
      
    l3=residual_layer(l2,64,(2,2))
    for i in range(17):
      l3=residual_layer(l3,64)
    
    
    l4=GlobalAveragePooling2D()(l3)
    outputs=Dense(classes,activation='softmax')(l4)
    
    return inputs,outputs
    
    

In [16]:
inputs,outputs=make_model()
resnet110=Model(inputs=inputs,outputs=outputs)
resnet110.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_112 (Conv2D)             (None, 32, 32, 16)   448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_112 (BatchN (None, 32, 32, 16)   64          conv2d_112[0][0]                 
__________________________________________________________________________________________________
activation_164 (Activation)     (None, 32, 32, 16)   0           batch_normalization_112[0][0]    
__________________________________________________________________________________________________
conv2d_113

add_78 (Add)                    (None, 16, 16, 32)   0           activation_233[0][0]             
                                                                 activation_235[0][0]             
__________________________________________________________________________________________________
activation_236 (Activation)     (None, 16, 16, 32)   0           add_78[0][0]                     
__________________________________________________________________________________________________
conv2d_162 (Conv2D)             (None, 16, 16, 32)   9248        activation_236[0][0]             
__________________________________________________________________________________________________
batch_normalization_162 (BatchN (None, 16, 16, 32)   128         conv2d_162[0][0]                 
__________________________________________________________________________________________________
activation_237 (Activation)     (None, 16, 16, 32)   0           batch_normalization_162[0][0]    
__________

In [0]:
optim=SGD(1e-2,momentum=0.9,decay=0.0001)
loss=categorical_crossentropy
resnet110.compile(optim,loss,metrics=['accuracy'])

In [19]:
resnet110.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/128,validation_steps=len(x_test)/128,epochs=50,verbose=1)

Epoch 1/50
373/390 [===========================>..] - ETA: 11s - loss: 2.4521 - acc: 0.2401390/390 [==============================] - 277s 710ms/step - loss: 2.4275 - acc: 0.2443 - val_loss: 1.9379 - val_acc: 0.3013
Epoch 2/50
390/390 [==============================] - 256s 656ms/step - loss: 1.8927 - acc: 0.3548 - val_loss: 1.8000 - val_acc: 0.3619
Epoch 3/50
 38/390 [=>............................] - ETA: 3:34 - loss: 1.8202 - acc: 0.3793390/390 [==============================] - 254s 651ms/step - loss: 1.7456 - acc: 0.4027 - val_loss: 1.8304 - val_acc: 0.3661
Epoch 4/50
328/390 [========================>.....] - ETA: 37s - loss: 1.6491 - acc: 0.4364390/390 [==============================] - 254s 651ms/step - loss: 1.6422 - acc: 0.4393 - val_loss: 1.5897 - val_acc: 0.4525
Epoch 5/50
390/390 [==============================] - 253s 648ms/step - loss: 1.5926 - acc: 0.4624 - val_loss: 1.7028 - val_acc: 0.4232
Epoch 6/50
 28/390 [=>............................] - ETA: 3:39 - loss: 1.5392 

In [0]:
resnet110.save_weights("./drive/ML/Cifar-10/weights_110_6426.h5")

In [0]:
resnet110.load_weights("./drive/ML/Cifar-10/weights_110_6426.h5")

In [0]:
import keras.backend as k

In [0]:
k.set_value(resnet110.optimizer.lr,0.001)

In [27]:
resnet110.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/128,validation_steps=len(x_test)/128,epochs=25,verbose=1)

Epoch 1/25
390/390 [==============================] - 253s 649ms/step - loss: 0.5524 - acc: 0.8323 - val_loss: 1.0247 - val_acc: 0.6903
Epoch 2/25
280/390 [====================>.........] - ETA: 1:06 - loss: 0.4839 - acc: 0.8563390/390 [==============================] - 253s 648ms/step - loss: 0.4816 - acc: 0.8565 - val_loss: 1.0204 - val_acc: 0.7002
Epoch 3/25
390/390 [==============================] - 253s 648ms/step - loss: 0.4391 - acc: 0.8682 - val_loss: 1.0264 - val_acc: 0.6933
Epoch 4/25
 17/390 [>.............................] - ETA: 3:44 - loss: 0.4337 - acc: 0.8658390/390 [==============================] - 253s 649ms/step - loss: 0.4159 - acc: 0.8764 - val_loss: 1.0379 - val_acc: 0.6960
Epoch 5/25
321/390 [=======================>......] - ETA: 41s - loss: 0.3976 - acc: 0.8794390/390 [==============================] - 253s 648ms/step - loss: 0.3988 - acc: 0.8797 - val_loss: 1.0307 - val_acc: 0.6982
Epoch 6/25
390/390 [==============================] - 253s 649ms/step - loss: 

In [0]:
resnet110.save_weights("./drive/ML/Cifar-10/weights_110_final_01.h5")